In [ ]:
from transformers import pipeline
import pandas as pd

## Zero Shot Classification
Load the dataset

In [ ]:
try:
    reviews_df = pd.read_csv("car_reviews.csv")
except FileNotFoundError:
    print("The file 'car_reviews.csv' was not found.")

Verify column names

In [ ]:
print(reviews_df.columns)

Clean column names

In [ ]:
reviews_df.columns = reviews_df.columns.str.strip().str.lower()
print(reviews_df.columns)

Initialize the zero shot classification pipeline and Define categories

In [ ]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


# Define categories
categories = [
    "talks about driving experience",
    "talks about features",
    "talks about value for money",
    "talks about issues",
    "other"
]

Apply the classificaion to each review

In [ ]:
# Function to classify a single review
def classify_review(review):
    result = classifier(review, candidate_labels=categories)
    return result['labels'][0]  # Top category

# Apply the classification function to the 'review' column
reviews_df['talks_about'] = reviews_df['review'].apply(classify_review)

Save the classified reviews to a new CSV file

In [ ]:
reviews_df.to_csv('intermediate_reviews_classified.csv', index=False)